# PyCaret Time Series — AirPassengers (univariate) and Delhi Climate (exogenous)

Datasets: `AirPassengers.csv` and `DailyDelhiClimateTrain.csv`.

## Environment
Using Python 3.11 with `pycaret==3.3.0` inside the shared `.venv`. Notebook assumes data CSVs are present in `../data/` and GPU is available; if not, PyCaret will fall back to CPU.

In [1]:
import pandas as pd
from pycaret.time_series import *

# Univariate: AirPassengers
air = pd.read_csv('../data/AirPassengers.csv')
air.columns = ['Month', 'Passengers']
air['Month'] = pd.to_datetime(air['Month'])
air = air.set_index('Month').asfreq('MS')
print(air.head())

            Passengers
Month                 
1949-01-01         112
1949-02-01         118
1949-03-01         132
1949-04-01         129
1949-05-01         121


In [2]:
s1 = setup(
    data=air['Passengers'],
    fh=24,
    fold=3,
    session_id=42,
    use_gpu=False,
    seasonal_period=12,
    log_experiment=True,
    experiment_name='pycaret_ts_air',
)
best_uni = compare_models()
tuned_uni = tune_model(best_uni)
plot_model(tuned_uni, plot='forecast', save=True)
final_uni = finalize_model(tuned_uni)
forecast_uni = predict_model(final_uni)
print(forecast_uni.head())
save_model(final_uni, 'ts_airpassengers_model')

,Description,Value
0,session_id,42
1,Target,Passengers
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(144, 1)"
5,Transformed data shape,"(144, 1)"
6,Transformed train set shape,"(119, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
exp_smooth,Exponential Smoothing,0.6917,0.7795,16.8629,21.3571,0.0540,0.0554,0.7730,0.0333
ets,ETS,0.6917,0.7796,16.8642,21.3586,0.0540,0.0554,0.7729,0.0400
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.7935,0.9281,19.6985,25.8296,0.0608,0.0630,0.6970,0.2967
arima,ARIMA,0.7945,0.8489,19.2302,23.0856,0.0627,0.0650,0.7210,0.0300
stlf,STLF,0.8177,0.8674,19.8339,23.6123,0.0646,0.0667,0.7096,0.0233
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.8219,0.9863,20.5084,27.7337,0.0627,0.0651,0.6653,0.3033
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.8262,0.9262,20.3099,25.5098,0.0648,0.0668,0.6867,0.1733
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.8522,0.9998,21.1527,27.9448,0.0657,0.0684,0.6523,0.1667
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.8604,1.0473,21.7348,29.7278,0.0656,0.0682,0.6243,0.5033
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.8775,1.0167,21.8258,28.4270,0.0685,0.0713,0.6384,0.3200


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,1952-11,nan,nan,nan,nan,nan,nan,nan
1,1954-11,0.6390,0.6854,14.6528,17.7144,0.0464,0.0471,0.8789
2,1956-11,1.1479,1.2035,33.5383,39.4596,0.0922,0.0867,0.5699
Mean,NaT,nan,nan,nan,nan,nan,nan,nan
SD,NaT,nan,nan,nan,nan,nan,nan,nan


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.5s finished


           y_pred
1961-01  445.2424
1961-02  418.2254
1961-03  465.3097
1961-04  494.9512
1961-05  505.4759
Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 ForecastingPipeline(steps=[('forecaster',
                                                                                             TransformedTargetForecaster(steps=[('model',
                                                                                                                                 ExponentialSmoothing(seasonal='mul',
                                                                                                                                                      sp=12,
                                                                                                                                                      trend='add'))]))]))]))]),
 'ts_airpassengers_model.pkl')

In [3]:
# Exogenous: Delhi Climate (meantemp target with other weather covariates)
climate = pd.read_csv('../data/DailyDelhiClimateTrain.csv')
climate['date'] = pd.to_datetime(climate['date'])
climate = climate.set_index('date').asfreq('D').interpolate()
exog = climate.drop(columns=['meantemp'])

s2 = setup(
    data=climate,
    target='meantemp',
    fh=30,
    fold=3,
    session_id=42,
    use_gpu=False,
    seasonal_period=7,
    enforce_exogenous=True,
    log_experiment=True,
    experiment_name='pycaret_ts_climate',
)
best_exog = compare_models()
tuned_exog = tune_model(best_exog)
plot_model(tuned_exog, plot='forecast', save=True)
plot_model(tuned_exog, plot='diagnostics', save=True)
final_exog = finalize_model(tuned_exog)
# Use recent exogenous values as placeholder for future horizon
future_exog = exog.tail(30)
forecast_exog = predict_model(final_exog, X=future_exog)
print(forecast_exog.head())
save_model(final_exog, 'ts_delhi_climate_model')


,Description,Value
0,session_id,42
1,Target,meantemp
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(1462, 4)"
5,Transformed data shape,"(1462, 4)"
6,Transformed train set shape,"(1431, 4)"
7,Transformed test set shape,"(30, 4)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.7098,0.6257,1.6909,1.8976,0.0644,0.0615,-0.7667,0.5200
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.7364,0.6431,1.7542,1.9504,0.0670,0.0640,-0.9716,0.4633
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.7562,0.6700,1.8010,2.0315,0.0683,0.0651,-0.9041,0.3200
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.7576,0.6508,1.8041,1.9733,0.0688,0.0661,-0.8501,0.7600
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,0.8037,0.7423,1.9178,2.2538,0.0725,0.0706,-2.9056,0.2367
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.9132,0.8437,2.1703,2.5528,0.0898,0.0817,-3.9415,0.2533
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.9461,0.8545,2.2534,2.5915,0.0859,0.0797,-2.0126,0.3967
arima,ARIMA,1.2255,1.0822,2.9187,3.2817,0.1109,0.1020,-3.9290,0.1367
auto_arima,Auto ARIMA,1.2319,1.0753,2.9318,3.2585,0.1138,0.1046,-3.9510,17.0300
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,1.2385,1.2259,2.9530,3.7227,0.1120,0.1077,-9.7140,0.3367


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2016-09-02,0.3044,0.2907,0.7326,0.8890,0.0230,0.0232,-0.1421
1,2016-10-02,1.2932,1.0648,3.0797,3.2302,0.1104,0.1039,-1.4870
2,2016-11-01,0.9373,0.8752,2.2212,2.6405,0.1015,0.0945,-2.6722
Mean,NaT,0.8450,0.7436,2.0112,2.2533,0.0783,0.0739,-1.4338
SD,NaT,0.4089,0.3295,0.9696,0.9942,0.0393,0.0361,1.0336


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.1s finished


In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.
In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.


             y_pred
2017-01-02  15.8283
2017-01-03  17.2544
2017-01-04  19.0104
2017-01-05  19.6750
2017-01-06  20.7832
Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 ForecastingPipeline(steps=[('forecaster',
                                                                                             TransformedTargetForecaster(steps=[('model',
                                                                                                                                 BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [7,
                                                                                                                                                                                                         6,
                                                                                                                                                                                                         5,
                          